# Deploy ML Model
https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html
https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#deploy-tensorflow-serving-models
https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/


After a TensorFlow estimator has been fit, it saves a TensorFlow SavedModel in the S3 location defined by output_path. You can call deploy on a TensorFlow estimator to create a SageMaker Endpoint, or you can call transformer to create a Transformer that you can use to run a batch transform job.

## AWS Setup

You will need to upload two files with no extension, "credentials" and "config".

The credentials file has your API key, config is preferred region

Put the following in each, change values to yours:
<hr>
credentials

```
[default]
aws_access_key_id = AKIAXN6R7JXYxxxxxxxx 
aws_secret_access_key = 1prI4W8J254zR5/ishEilmqskaZLOnazxxxxxxxx
```
<hr>
config

```
[default]
region = us-east-1
```

In [1]:
!apt-get update -q >apt-get_install.og
!pip3 -q install awscli
!pip3 -q install boto3
!pip3 -q install sagemaker
!pip3 -q install opendatasets
!pip freeze >requirements.txt
import boto3
from google.colab import files
import os, sys, stat
import shutil

# Set the bucket name that will be used everywhere
aws_bucket = '511004593648-msds436'

# check if AWS credentials already loaded, if not prompt for upload
aws_folder = os.path.expanduser('~/.aws')
path_to_file = f'{aws_folder}/credentials'
print("AWS credentials location:", path_to_file)

if os.path.exists(path_to_file):
  print("AWS credentials found in ~/.aws/credentials")
else:
  print('-'*80)
  print("Upload your AWS credentials and config files:")
  files_uploaded = files.upload()
  os.makedirs(aws_folder, exist_ok=True)
  shutil.copy('credentials',f'{aws_folder}/credentials')
  shutil.copy('credentials',f'{aws_folder}/config')
  os.chmod(f'{aws_folder}/credentials', stat.S_IRWXU)
  os.chmod(f'{aws_folder}/config', stat.S_IRWXU)
  print("files uploaded: ",files_uploaded.keys)

# all ok? this command should work
comprehend = boto3.client(service_name='comprehend', region_name="us-east-1")
text = "Machine learning will automate jobs that most people thought could only be done by people." #~Dave Waters
print("text to perform sentiment analysis: \n",text)
comprehend.detect_sentiment(Text=text, LanguageCode='en')

     |████████████████████████████████| 3.8 MB 10.0 MB/s 
     |████████████████████████████████| 547 kB 62.3 MB/s 
     |████████████████████████████████| 79 kB 6.8 MB/s 
     |████████████████████████████████| 8.6 MB 39.7 MB/s 
     |████████████████████████████████| 138 kB 47.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 132 kB 8.7 MB/s 
     |████████████████████████████████| 513 kB 7.5 MB/s 
     |████████████████████████████████| 49 kB 4.7 MB/s 
     |████████████████████████████████| 81 kB 8.3 MB/s 
     |████████████████████████████████| 65 kB 3.4 MB/s 
ERROR: pip's dependency resolver does not curr

Saving config to config
Saving credentials to credentials
files uploaded:  <built-in method keys of dict object at 0x7fb9e3311d70>
text to perform sentiment analysis: 
 Machine learning will automate jobs that most people thought could only be done by people.


{'ResponseMetadata': {'HTTPHeaders': {'content-length': '162',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 12 Mar 2022 17:11:04 GMT',
   'x-amzn-requestid': 'd4a7ce5c-b868-437b-beec-632083139462'},
  'HTTPStatusCode': 200,
  'RequestId': 'd4a7ce5c-b868-437b-beec-632083139462',
  'RetryAttempts': 0},
 'Sentiment': 'NEUTRAL',
 'SentimentScore': {'Mixed': 0.009578169323503971,
  'Negative': 0.01551244780421257,
  'Neutral': 0.8685871362686157,
  'Positive': 0.10632215440273285}}

## Setup

In [12]:
!pip install "h5py==2.10.0"
import h5py
import numpy as np
import boto3
import sagemaker
import tensorflow as tf

# reference: https://github.com/keras-team/keras/issues/14265
print(tf.__version__) 
tf_framework_version = tf.__version__

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
boto3_sess = boto3.Session(region_name="us-east-1")
conn = boto3.client(service_name='sagemaker' )
sess = sagemaker.session.Session()
bucket = sess.default_bucket()

print(conn.meta)

2.8.0


## Deploy TensorFlow model to an Amazon SageMaker
The model stored as tar file on S3 is deployed to an instance and creates endpoint.  Note the role needs to be created on AWS to include the SageMaker full access.

- https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/deploying_tensorflow_serving.html
- https://sagemaker.readthedocs.io/en/stable/overview.html#byo-model
- https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/deploying_tensorflow_serving.html#deploying-directly-from-model-artifacts

In [25]:
from sagemaker.tensorflow import TensorFlowModel
from sagemaker.model import Model

# see https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html
image_uri = '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.8.0-gpu-py39-cu112-ubuntu20.04-e3'
model = Model(image_uri,
              model_data='s3://511004593648-msds436/mask_model.tar.gz',
              role='sagemaker-role')

predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.c5.large' )

--------------------------------------------------------------------*

UnexpectedStatusException: ignored

## Batch Transformation

From https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#run-a-batch-transform-job 

and https://sagemaker.readthedocs.io/en/stable/overview.html#sagemaker-batch-transform

Use the inference endpoint and run over an input directory.

https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_runtime_InvokeEndpoint.html 

In [ ]:
# Create folder in the S3 bucket where the results are stored

# list contents of input folder

# for each image, send to endpoint and capture response, move to result folder

# complile results